In [ ]:
import h5py
from os.path import join
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import progressbar
import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from mssa_learning.models.dnn_classifier import DNNClassifier
from mssa_learning.models.rnn_classifier import RNNClassifier
from mssa_learning.models.lstm_classifier import LSTMClassifier
from mssa_learning.tools.dataloader import *

from sklearn.svm import LinearSVC
import csv

In [ ]:
init_notebook_mode(connected=True)

### Load dataset

In [ ]:
h5_data = h5py.File(join("..", "data", "dataset_converted" , "h5", "pc_kalman.h5"))

In [ ]:
features, labels = h5_to_numpy(h5_data, nb_components=10)

In [ ]:
train_x, train_y, test_x, test_y = create_training_base(features, labels, 5)

### RNN Classification

In [ ]:
D_in = len(train_x[0][0])
H = 64
D_out = max(train_y) + 1  # number of classes to learn

In [ ]:
steps = 100000
epochs = 1

In [ ]:
model = RNNClassifier(D_in, D_out, H, steps=steps, epochs=epochs, scale=False)

In [ ]:
model.fit(train_x, train_y)

In [ ]:
nb_errors = 0
for i, input_data in enumerate(test_x):
    model.hidden = model.init_hidden()
    x = torch.zeros(len(input_data), 1, len(input_data[0]))
    for row in range(len(input_data)):
        x[row, 0] = torch.from_numpy(input_data[row]).float()
    x = Variable(x)
    for j in range(x.size()[0]):
        output = model.forward(x[j])
    top_n, top_i = output.data.topk(1)
    y_pred = int(top_i[0][0])

    if y_pred != test_y[i]:
        nb_errors += 1
success_rate = (len(test_x) - nb_errors) / float(len(test_x))
print success_rate

### LSTM Classification

In [ ]:
D_in = len(train_x[0][0])
H = 32
D_out = max(train_y)  # number of classes to learn

In [ ]:
steps = 50000
epochs = 1

In [ ]:
model = LSTMClassifier(D_in, D_out, H, steps=steps, epochs=epochs, scale=False)

In [ ]:
model.fit(train_x, train_y)

In [ ]:
nb_errors = 0
for i, input_data in enumerate(test_x):
    model.hidden = model.init_hidden()
    x = torch.zeros(len(input_data), 1, len(input_data[0]))
    for row in range(len(input_data)):
        x[row, 0] = torch.from_numpy(input_data[row]).float()
    x = Variable(x)
    output = model.forward(x)
    top_n, top_i = output.data.topk(1)
    y_pred = int(top_i[0][0])
    if y_pred != test_y[i]:
        nb_errors += 1
success_rate = (len(test_x) - nb_errors) / float(len(test_x))
print success_rate

#### CSV export

In [ ]:
def data_to_rows(features, labels):
    rows = np.hstack((features, labels))
    return rows

In [ ]:
with open('/tmp/train.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerows(rows)

In [ ]:
h5_pc = h5py.File(join("..", "data", "dataset_converted" , "h5", "kalman.h5"))

In [ ]:
h5_pc['P001R001A001'].keys()